# 🤖 Restaurant Intelligence Chatbot - Production System

**Enterprise-Grade Conversational AI with RAG**

This system integrates:
- ✅ Advanced Sentiment Analysis
- ✅ Independent Aspect-Based Analysis
- ✅ Vector Retrieval (RAG)
- ✅ LLM-Driven Recommendations
- ✅ Hallucination Prevention
- ✅ Production-Grade Error Handling

---

## System Architecture

```
User Query → Intent Router → Vector Retrieval → LLM Reasoning → Structured Response
                ↓                    ↓                ↓
         Context Memory      Semantic Search    Grounded Generation
```

In [1]:
!pip install -q \
    chromadb \
    langchain \
    langchain-community \
    sentence-transformers \
    transformers \
    accelerate \
    python-dotenv



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.0/52.0 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.4/21.4 MB 87.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 95.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 24.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 79.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 74.9 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.6/132.6 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.0/220.0 kB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.5/66.5 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━

## 📦 Installation & Dependencies

## ⚙️ Configuration & Environment Setup

In [2]:
# Core Imports
import os
import warnings
import logging
from pathlib import Path
from typing import Dict, List, Optional, Tuple, Any
from dataclasses import dataclass, field
import json

# Data Processing
import pandas as pd
import numpy as np
from tqdm import tqdm

# ML & NLP
import torch
from transformers import pipeline
from sentence_transformers import SentenceTransformer

# Vector DB & RAG
import chromadb
from chromadb.config import Settings
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.schema import Document
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory
from langchain.llms.base import LLM

# Validation
from pydantic import BaseModel, Field, validator

# Configuration
warnings.filterwarnings('ignore')
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)
tqdm.pandas()

print("✅ All dependencies loaded successfully")

2026-02-15 15:59:00.048478: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1771171140.275139      55 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1771171140.340010      55 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1771171140.873177      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1771171140.873230      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1771171140.873233      55 computation_placer.cc:177] computation placer alr

✅ All dependencies loaded successfully


In [3]:
@dataclass
class SystemConfig:
    """Production-grade configuration with environment variable support"""
    
    # Paths
    data_path: str = "/kaggle/input/datasets/shahriard07/restaurant-review/dhaka_restaurants.csv"
    vector_db_path: str = "/kaggle/working/restaurant_vector_db"
    
    # Model Configuration
    sentiment_model: str = "cardiffnlp/twitter-xlm-roberta-base-sentiment"
    embedding_model: str = "all-MiniLM-L6-v2"
    device: str = "cuda" if torch.cuda.is_available() else "cpu"
    
    # LLM Configuration
    llm_temperature: float = 0.2
    max_tokens: int = 512
    
    # RAG Configuration
    retrieval_k: int = 5
    similarity_threshold: float = 0.7
    
    # Aspect Keywords
    aspects: Dict[str, List[str]] = field(default_factory=lambda: {
        "food": ["food", "taste", "meal", "dish", "cuisine", "flavor", "delicious", "খাবার", "স্বাদ"],
        "service": ["service", "staff", "waiter", "waitress", "manager", "server", "সার্ভিস", "কর্মী"],
        "price": ["price", "cost", "expensive", "cheap", "value", "affordable", "দাম", "মূল্য"],
        "ambience": ["ambience", "atmosphere", "environment", "decor", "vibe", "পরিবেশ"],
        "cleanliness": ["clean", "hygiene", "sanitary", "dirty", "পরিষ্কার"]
    })
    
    # Negative Triggers
    negative_triggers: List[str] = field(default_factory=lambda: [
        "late", "slow", "rude", "bad", "cold", "delay", "terrible", "awful",
        "disappointing", "poor", "worst", "খারাপ", "দেরি", "ঠান্ডা"
    ])
    
    # Production Settings
    batch_size: int = 32
    enable_logging: bool = True
    fallback_enabled: bool = True

# Initialize configuration
config = SystemConfig()
logger.info(f"System initialized on device: {config.device}")
print(f"🔧 Configuration loaded - Device: {config.device}")

🔧 Configuration loaded - Device: cuda


## 📊 Data Layer - Schema Validation & Cleaning

In [4]:
class ReviewSchema(BaseModel):
    """Pydantic schema for review validation"""
    business_name: str = Field(..., min_length=1)
    review_text: str = Field(..., min_length=10)
    review_rating: float = Field(..., ge=1.0, le=5.0)
    business_address: Optional[str] = None
    
    @validator('review_text')
    def validate_text(cls, v):
        if not isinstance(v, str) or len(v.strip()) < 10:
            raise ValueError('Review text must be at least 10 characters')
        return v.strip()

class DataPipeline:
    """Production-grade data processing pipeline"""
    
    def __init__(self, config: SystemConfig):
        self.config = config
        self.df_raw = None
        self.df_cleaned = None
        
    def load_and_validate(self, path: str) -> pd.DataFrame:
        """Load data with schema validation"""
        try:
            logger.info(f"Loading data from {path}")
            df = pd.read_csv(path)
            self.df_raw = df.copy()
            
            # Validate required columns
            required_cols = ['business_name', 'review_text', 'review_rating']
            missing = set(required_cols) - set(df.columns)
            if missing:
                raise ValueError(f"Missing required columns: {missing}")
            
            logger.info(f"✅ Loaded {len(df)} rows")
            return df
            
        except Exception as e:
            logger.error(f"Data loading failed: {e}")
            raise
    
    def clean_data(self, df: pd.DataFrame) -> pd.DataFrame:
        """Clean and normalize data"""
        logger.info("Starting data cleaning pipeline")
        
        # Drop nulls
        df_clean = df.dropna(subset=['review_text', 'business_name']).copy()
        logger.info(f"Removed {len(df) - len(df_clean)} null rows")
        
        # Ensure text is string
        df_clean = df_clean[df_clean['review_text'].apply(lambda x: isinstance(x, str))]
        
        # Text normalization
        df_clean['review_text'] = (
            df_clean['review_text']
            .str.strip()
            .str.replace(r'\s+', ' ', regex=True)
        )
        
        # Filter short reviews
        df_clean = df_clean[df_clean['review_text'].str.len() >= 10]
        
        # Normalize restaurant names
        df_clean['business_name_normalized'] = (
            df_clean['business_name']
            .str.strip()
            .str.lower()
        )
        
        # Deduplicate
        before_dedup = len(df_clean)
        df_clean = df_clean.drop_duplicates(subset=['business_name', 'review_text'])
        logger.info(f"Removed {before_dedup - len(df_clean)} duplicate reviews")
        
        self.df_cleaned = df_clean.reset_index(drop=True)
        logger.info(f"✅ Cleaning complete - {len(self.df_cleaned)} clean rows")
        
        return self.df_cleaned
    
    def get_restaurant_index(self) -> Dict[str, int]:
        """Create restaurant name index for fast lookup"""
        if self.df_cleaned is None:
            raise ValueError("Data not cleaned yet")
        
        return (
            self.df_cleaned
            .groupby('business_name_normalized')
            .size()
            .to_dict()
        )

# Initialize and run data pipeline
data_pipeline = DataPipeline(config)
df = data_pipeline.load_and_validate(config.data_path)
df_cleaned = data_pipeline.clean_data(df)

print(f"\n📊 Data Summary:")
print(f"Total Reviews: {len(df_cleaned):,}")
print(f"Unique Restaurants: {df_cleaned['business_name'].nunique():,}")
print(f"Average Review Length: {df_cleaned['review_text'].str.len().mean():.0f} chars")
df_cleaned.head(3)


📊 Data Summary:
Total Reviews: 977
Unique Restaurants: 126
Average Review Length: 405 chars


,business_name,business_address,business_phone,business_website,business_rating,business_total_reviews,reviewer_name,review_rating,review_date,review_text,review_additional_info,business_name_normalized
0,Izumi Japanese Kitchen,"House 24 C, Rd 119, Dhaka 1212, Bangladesh",+880 1933-446677,https://m.facebook.com/izumiBD/,4.5,2233,"{'name': 'Raunak Maskay', 'thumbnail': 'https:...",5.0,a month ago,"Izumi Japanese Kitchen in Gulshan, Dhaka is on...",NaN,izumi japanese kitchen
1,Izumi Japanese Kitchen,"House 24 C, Rd 119, Dhaka 1212, Bangladesh",+880 1933-446677,https://m.facebook.com/izumiBD/,4.5,2233,"{'name': 'Dewan Asif', 'thumbnail': 'https://l...",5.0,4 months ago,Izumi Japanese Kitchen is a great place for re...,NaN,izumi japanese kitchen
2,Izumi Japanese Kitchen,"House 24 C, Rd 119, Dhaka 1212, Bangladesh",+880 1933-446677,https://m.facebook.com/izumiBD/,4.5,2233,"{'name': 'Dr. Mehruba Mona', 'thumbnail': 'htt...",5.0,Edited 8 months ago,One of the authentic Japanese restaurant in Dh...,NaN,izumi japanese kitchen


## 🎭 Sentiment Engine - Advanced Analysis with Fallback

In [5]:
class SentimentEngine:
    """Production sentiment analyzer with fallback mechanisms"""
    
    def __init__(self, config: SystemConfig):
        self.config = config
        self.model = None
        self.fallback_mode = False
        self._load_model()
    
    def _load_model(self):
        """Load sentiment model with retry logic"""
        try:
            logger.info(f"Loading sentiment model: {self.config.sentiment_model}")
            self.model = pipeline(
                "sentiment-analysis",
                model=self.config.sentiment_model,
                device=0 if self.config.device == "cuda" else -1
            )
            logger.info("✅ Sentiment model loaded")
        except Exception as e:
            logger.warning(f"Model loading failed: {e}. Enabling fallback mode.")
            self.fallback_mode = True
    
    def _normalize_label(self, label: str) -> str:
        """Normalize sentiment labels to standard format"""
        label_lower = label.lower()
        if 'pos' in label_lower:
            return 'positive'
        elif 'neg' in label_lower:
            return 'negative'
        else:
            return 'neutral'
    
    def _fallback_sentiment(self, text: str) -> Tuple[str, float]:
        """Rule-based fallback sentiment analysis"""
        text_lower = text.lower()
        
        positive_words = ['good', 'great', 'excellent', 'amazing', 'love', 'best', 'wonderful']
        negative_words = ['bad', 'terrible', 'awful', 'worst', 'hate', 'poor', 'disappointing']
        
        pos_count = sum(1 for word in positive_words if word in text_lower)
        neg_count = sum(1 for word in negative_words if word in text_lower)
        
        if pos_count > neg_count:
            return 'positive', 0.6
        elif neg_count > pos_count:
            return 'negative', 0.6
        else:
            return 'neutral', 0.5
    
    def analyze(self, text: str) -> Tuple[str, float]:
        """Analyze sentiment with error handling"""
        try:
            if self.fallback_mode or self.model is None:
                return self._fallback_sentiment(text)
            
            # Truncate to model limit
            result = self.model(text[:512])[0]
            label = self._normalize_label(result['label'])
            confidence = result['score']
            
            return label, confidence
            
        except Exception as e:
            logger.warning(f"Sentiment analysis failed for text, using fallback: {e}")
            return self._fallback_sentiment(text)
    
    def batch_analyze(self, texts: List[str]) -> pd.DataFrame:
        """Batch process with progress tracking"""
        logger.info(f"Analyzing {len(texts)} reviews")
        
        results = []
        for text in tqdm(texts, desc="Sentiment Analysis"):
            label, confidence = self.analyze(text)
            results.append({'sentiment': label, 'confidence': confidence})
        
        return pd.DataFrame(results)

# Run sentiment analysis
sentiment_engine = SentimentEngine(config)
sentiment_results = sentiment_engine.batch_analyze(df_cleaned['review_text'].tolist())

df_cleaned['overall_sentiment'] = sentiment_results['sentiment']
df_cleaned['sentiment_confidence'] = sentiment_results['confidence']

print("\n🎭 Sentiment Distribution:")
print(df_cleaned['overall_sentiment'].value_counts())
print(f"\nAverage Confidence: {df_cleaned['sentiment_confidence'].mean():.2%}")

config.json:   0%|          | 0.00/841 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

Device set to use cuda:0
Sentiment Analysis: 100%|██████████| 977/977 [00:10<00:00, 96.86it/s] 


🎭 Sentiment Distribution:
overall_sentiment
positive    726
negative    213
neutral      38
Name: count, dtype: int64

Average Confidence: 72.86%


## 🔍 Aspect Extraction - Independent Analysis (NOT Copied from Overall)

In [6]:
# ==============================
# Aspect Extraction Module
# ==============================

class AspectAnalyzer:
    """Advanced aspect-based sentiment analyzer (Production Safe)"""

    def __init__(self, config: SystemConfig, sentiment_engine: SentimentEngine):
        self.config = config
        self.sentiment_engine = sentiment_engine
        self.aspects = config.aspects

    def extract_aspect_text(self, text: str, aspect: str) -> Optional[str]:
        if not isinstance(text, str) or not text.strip():
            return None

        text_lower = text.lower()
        keywords = self.aspects.get(aspect, [])

        # Aspect not mentioned at all
        if not any(keyword in text_lower for keyword in keywords):
            return None

        # Extract relevant sentences
        sentences = text.split(".")
        relevant = []

        for sentence in sentences:
            if any(keyword in sentence.lower() for keyword in keywords):
                relevant.append(sentence.strip())

        return " ".join(relevant) if relevant else text[:200]

    def analyze_aspect(self, text: str, aspect: str) -> Dict[str, Any]:
        aspect_text = self.extract_aspect_text(text, aspect)

        if aspect_text is None:
            return {
                "mentioned": False,
                "sentiment": None,
                "confidence": 0.0
            }

        sentiment, confidence = self.sentiment_engine.analyze(aspect_text)

        return {
            "mentioned": True,
            "sentiment": sentiment,
            "confidence": confidence
        }

    def batch_analyze(self, texts: List[str]) -> pd.DataFrame:
        results = []

        for text in tqdm(texts, desc="Aspect Analysis"):
            row = {}

            for aspect in self.aspects.keys():
                data = self.analyze_aspect(text, aspect)

                row[f"{aspect}_mentioned"] = bool(data["mentioned"])
                row[f"{aspect}_sentiment"] = data["sentiment"]
                row[f"{aspect}_confidence"] = float(data["confidence"])

            results.append(row)

        return pd.DataFrame(results)


# ==============================
# Run Aspect Analysis
# ==============================

aspect_analyzer = AspectAnalyzer(config, sentiment_engine)

aspect_results = aspect_analyzer.batch_analyze(
    df_cleaned["review_text"].fillna("").tolist()
)

# ------------------------------
# Remove previous aspect columns (if re-run)
# ------------------------------

aspect_prefixes = list(config.aspects.keys())

cols_to_drop = [
    col for col in df_cleaned.columns
    if any(col.startswith(prefix) for prefix in aspect_prefixes)
]

df_cleaned = df_cleaned.drop(columns=cols_to_drop, errors="ignore")

# ------------------------------
# Safe Merge
# ------------------------------

df_cleaned = pd.concat(
    [
        df_cleaned.reset_index(drop=True),
        aspect_results.reset_index(drop=True)
    ],
    axis=1
)

# ==============================
# Safe Aspect Summary
# ==============================

print("\n🔍 Aspect Analysis Summary:")

for aspect in config.aspects.keys():

    col_mentioned = f"{aspect}_mentioned"
    col_sentiment = f"{aspect}_sentiment"

    if col_mentioned not in df_cleaned.columns:
        continue

    mask = df_cleaned[col_mentioned] == True
    mentioned_count = int(mask.sum())

    if mentioned_count > 0:

        sentiment_dist = (
            df_cleaned.loc[mask, col_sentiment]
            .dropna()
            .value_counts()
        )

        print(f"\n{aspect.upper()} → {mentioned_count} mentions")
        print(sentiment_dist)


Aspect Analysis: 100%|██████████| 977/977 [00:17<00:00, 55.11it/s]


🔍 Aspect Analysis Summary:

FOOD → 837 mentions
food_sentiment
positive    595
negative    177
neutral      65
Name: count, dtype: int64

SERVICE → 508 mentions
service_sentiment
positive    400
negative     88
neutral      20
Name: count, dtype: int64

PRICE → 306 mentions
price_sentiment
positive    130
negative    113
neutral      63
Name: count, dtype: int64

AMBIENCE → 388 mentions
ambience_sentiment
positive    328
negative     41
neutral      19
Name: count, dtype: int64

CLEANLINESS → 74 mentions
cleanliness_sentiment
positive    55
negative    13
neutral      6
Name: count, dtype: int64


## ⚠️ Conflict Detection - Multi-Level Analysis

In [7]:
class ConflictDetector:
    """Advanced conflict detection system"""
    
    def detect_rating_sentiment_conflict(self, row: pd.Series) -> str:
        rating = row.get('review_rating', 0)
        sentiment = row.get('overall_sentiment')
        
        if rating >= 4 and sentiment == 'negative':
            return 'Hidden Dissatisfaction'
        elif rating <= 2 and sentiment == 'positive':
            return 'Politeness Bias'
        elif rating == 3 and sentiment in ['positive', 'negative']:
            return 'Ambiguous Experience'
        else:
            return 'No Conflict'
    
    def detect_aspect_conflicts(self, row: pd.Series, aspects: List[str]) -> int:
        overall = row.get('overall_sentiment')
        conflicts = 0
        
        for aspect in aspects:
            if row.get(f'{aspect}_mentioned', False):
                aspect_sent = row.get(f'{aspect}_sentiment')
                if aspect_sent and aspect_sent != overall:
                    conflicts += 1
        
        return conflicts
    
    def analyze(self, df: pd.DataFrame, aspects: List[str]) -> pd.DataFrame:
        logger.info("Running conflict detection")
        
        df['rating_sentiment_conflict'] = df.apply(
            self.detect_rating_sentiment_conflict, axis=1
        )
        
        df['aspect_conflict_count'] = df.apply(
            lambda row: self.detect_aspect_conflicts(row, aspects), axis=1
        )
        
        df['has_conflict'] = (
            (df['rating_sentiment_conflict'] != 'No Conflict') |
            (df['aspect_conflict_count'] > 0)
        )
        
        logger.info("✅ Conflict detection complete")
        return df

conflict_detector = ConflictDetector()
df_cleaned = conflict_detector.analyze(df_cleaned, list(config.aspects.keys()))

print("\n⚠️ Conflict Analysis:")
print(df_cleaned['rating_sentiment_conflict'].value_counts())
print(f"\nTotal Conflicts: {df_cleaned['has_conflict'].sum():,} ({df_cleaned['has_conflict'].mean():.1%})")


⚠️ Conflict Analysis:
rating_sentiment_conflict
No Conflict               799
Ambiguous Experience       87
Hidden Dissatisfaction     82
Politeness Bias             9
Name: count, dtype: int64

Total Conflicts: 364 (37.3%)


## Text Chunking for RAG

In [8]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(
    chunk_size=400,
    chunk_overlap=50
)

def chunk_reviews(df):
    new_rows = []

    for _, row in df.iterrows():
        chunks = splitter.split_text(row["review_text"])

        for chunk in chunks:
            new_row = row.copy()
            new_row["review_text"] = chunk
            new_rows.append(new_row)

    return pd.DataFrame(new_rows)

df_chunked = chunk_reviews(df_cleaned)


## 🗄️ Vector Database - RAG Layer

In [9]:
import shutil

class VectorStoreManager:
    """Production vector database manager"""
    
    def __init__(self, config: SystemConfig):
        self.config = config
        self.embeddings = None
        self.vector_store = None
        self._initialize_embeddings()
    
    def _initialize_embeddings(self):
        try:
            logger.info(f"Loading embedding model: {self.config.embedding_model}")
            self.embeddings = HuggingFaceEmbeddings(
                model_name=self.config.embedding_model
            )
            logger.info("✅ Embedding model loaded")
        except Exception as e:
            logger.error(f"Embedding model loading failed: {e}")
            raise
    
    def create_documents(self, df: pd.DataFrame) -> List[Document]:
        logger.info(f"Creating {len(df)} documents")
        documents = []
        
        for _, row in df.iterrows():
            aspect_summary = {}
            for aspect in self.config.aspects.keys():
                if row.get(f'{aspect}_mentioned', False):
                    aspect_summary[aspect] = row.get(f'{aspect}_sentiment')
            
            doc = Document(
                page_content=row['review_text'],
                metadata={
                    'restaurant': row['business_name'],
                    'restaurant_normalized': row['business_name_normalized'],
                    'rating': float(row['review_rating']),
                    'sentiment': row['overall_sentiment'],
                    'confidence': float(row['sentiment_confidence']),
                    'aspects': json.dumps(aspect_summary),
                    'conflict': row['rating_sentiment_conflict'],
                    'has_conflict': bool(row['has_conflict'])
                }
            )
            documents.append(doc)
        
        return documents
    
    def build_vector_store(self, documents: List[Document]) -> Chroma:
        try:
            if os.path.exists(self.config.vector_db_path):
                shutil.rmtree(self.config.vector_db_path)
            
            logger.info("Building vector store...")
            self.vector_store = Chroma.from_documents(
                documents=documents,
                embedding=self.embeddings,
                persist_directory=self.config.vector_db_path,
                client_settings=Settings(anonymized_telemetry=False)
            )
            
            logger.info("✅ Vector store built")
            return self.vector_store
        except Exception as e:
            logger.error(f"Vector store creation failed: {e}")
            raise

vector_manager = VectorStoreManager(config)
documents = vector_manager.create_documents(df_chunked)
vector_store = vector_manager.build_vector_store(documents)

print(f"\n🗄️ Vector Store Ready - {len(documents):,} documents")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]


🗄️ Vector Store Ready - 1,541 documents


### CELL 9: LLM Provider Layer ###

In [ ]:
import google.generativeai as genai
import os

# API key configure
genai.configure(api_key="YOUR_API_KEY_HERE")

class GeminiLLM:
    def __init__(self, temperature=0.0):
        self.model = genai.GenerativeModel("models/gemini-2.5-flash")
        self.temperature = temperature

    def __call__(self, prompt: str):
        response = self.model.generate_content(
            prompt,
            generation_config={
                "temperature": 0.0,
                "top_p": 0.8
            }
        )
        return response.text


### CELL 10: RAG Chatbot ###

In [29]:
import re
import numpy as np

class RAGChatbot:

    def __init__(self, vector_store, llm, config, df):
        self.vector_store = vector_store
        self.llm = llm
        self.config = config
        self.df = df
        self.memory = []
        self.recommendation_engine = RecommendationEngine(df)

    # ---------------------------
    # Intent Detection
    # ---------------------------
    def classify_query(self, question):
        q = question.lower()

        if re.search(r"top\s*\d+", q):
            return "top_n"

        if "compare" in q:
            return "comparison"

        if "best" in q or "rank" in q:
            return "ranking"

        if "recommend" in q:
            return "recommendation"

        return "general"

    # ---------------------------
    # Extract Top N
    # ---------------------------
    def extract_top_n(self, question):
        match = re.search(r"top\s*(\d+)", question.lower())
        if match:
            return int(match.group(1))
        return None

    # ---------------------------
    # Extract Restaurant Entities
    # ---------------------------
    def extract_entities(self, question):
        q = question.lower()
        entities = []

        for name in self.df["business_name_normalized"].unique():
            if name in q:
                entities.append(name)

        return entities

    # ---------------------------
    # Dataset-wide Ranking
    # ---------------------------
    def rank_restaurants(self, top_n=5, focus="overall"):

        results = []

        for r in self.df["business_name_normalized"].unique():
            score_data = self.recommendation_engine.calculate_score(r)

            if "error" not in score_data:
                results.append(score_data)

        ranked = sorted(results, key=lambda x: x["total_score"], reverse=True)

        return ranked[:top_n]

    # ---------------------------
    # Multi-Restaurant Comparison
    # ---------------------------
    def compare_restaurants(self, entities):

        comparisons = []

        for e in entities:
            score_data = self.recommendation_engine.calculate_score(e)
            if "error" not in score_data:
                comparisons.append(score_data)

        if len(comparisons) < 2:
            return None

        ranked = sorted(comparisons, key=lambda x: x["total_score"], reverse=True)

        return ranked

    # ---------------------------
    # Human-Friendly Presentation Layer
    # ---------------------------
    def present_ranking(self, ranked_list):

        text = "Here are the strongest performers based on your dataset:\n\n"

        for i, r in enumerate(ranked_list, 1):
            text += f"{i}. {r['restaurant']} (Score: {r['total_score']})\n"

        text += "\nThese rankings are calculated using sentiment strength, rating consistency, volume, and confidence."

        return text

    def present_comparison(self, ranked_list):

        text = "Here’s a clear comparison based on your dataset:\n\n"

        for r in ranked_list:
            text += (
                f"{r['restaurant']} → "
                f"Score: {r['total_score']}, "
                f"Avg Rating: {r['average_rating']}, "
                f"Reviews: {r['review_count']}\n"
            )

        winner = ranked_list[0]["restaurant"]

        text += f"\nOverall, {winner} currently shows the strongest performance based on available data."

        return text

    # ---------------------------
    # General RAG Mode
    # ---------------------------
    def rag_answer(self, question):

        docs = self.vector_store.similarity_search(question, k=self.config.retrieval_k)

        if not docs:
            return {
                "answer": "I don’t see enough relevant review evidence for that specific question.",
                "confidence": 0.0
            }

        context = "\n".join([doc.page_content for doc in docs])

        prompt = f"""
You are a restaurant analytics consultant.

Answer naturally and clearly.
Avoid academic tone.
Use only provided evidence.

Evidence:
{context}

Question:
{question}
"""

        answer = self.llm(prompt)

        return {
            "answer": answer,
            "confidence": 0.85
        }

    # ---------------------------
    # Main Query Router
    # ---------------------------
    def query(self, question, restaurant_filter=None):

        intent = self.classify_query(question)

        # -------- Top N Mode --------
        if intent == "top_n":
            top_n = self.extract_top_n(question) or 5
            ranked = self.rank_restaurants(top_n)
            answer = self.present_ranking(ranked)

            return {
                "answer": answer,
                "confidence": 0.95,
                "intent": intent
            }

        # -------- Comparison Mode --------
        entities = self.extract_entities(question)

        if intent == "comparison" and len(entities) >= 2:
            ranked = self.compare_restaurants(entities)

            if ranked:
                answer = self.present_comparison(ranked)

                return {
                    "answer": answer,
                    "confidence": 0.93,
                    "intent": intent
                }

        # -------- General RAG Mode --------
        rag_response = self.rag_answer(question)
        rag_response["intent"] = "general"

        return rag_response


### CELL 11: Recommendation Engine ###

In [30]:
class RecommendationEngine:
    def __init__(self, df: pd.DataFrame):
        self.df = df

    def calculate_score(self, restaurant_name: str):

        df_rest = self.df[
            self.df["business_name_normalized"] == restaurant_name.lower()
        ]

        if df_rest.empty:
            return {"error": "Restaurant not found"}

        if len(df_rest) < 5:
            return {"error": "Not enough review volume for reliable ranking"}

        # =============================
        # Sentiment Score (40%)
        # =============================
        sentiment_score = (df_rest["overall_sentiment"] == "positive").mean() * 40

        # =============================
        # Aspect Score (25%)
        # =============================
        aspect_cols = [c for c in df_rest.columns if "_sentiment" in c]

        aspect_score_total = 0
        aspect_count = 0

        for col in aspect_cols:
            mentioned = df_rest[df_rest[col.replace("_sentiment","_mentioned")] == True]
            if not mentioned.empty:
                pos_rate = (mentioned[col] == "positive").mean()
                aspect_score_total += pos_rate
                aspect_count += 1

        aspect_score = (aspect_score_total / aspect_count) * 25 if aspect_count > 0 else 0

        # =============================
        # Conflict Penalty (15%)
        # =============================
        conflict_rate = df_rest["has_conflict"].mean()
        conflict_score = (1 - conflict_rate) * 15

        # =============================
        # Volume Bonus (10%)
        # =============================
        volume_score = min(len(df_rest) / 100, 1.0) * 10

        # =============================
        # Confidence Bonus (10%)
        # =============================
        confidence_score = df_rest["sentiment_confidence"].mean() * 10

        total_score = (
            sentiment_score +
            aspect_score +
            conflict_score +
            volume_score +
            confidence_score
        )

        return {
            "restaurant": restaurant_name,
            "total_score": round(total_score, 2),
            "review_volume": len(df_rest),
            "sentiment_positive_ratio": round((df_rest["overall_sentiment"] == "positive").mean(), 2),
            "conflict_rate": round(conflict_rate, 2)
        }


In [32]:
llm = GeminiLLM()
chatbot = RAGChatbot(vector_store, llm, config, df_cleaned)



In [14]:
# --- Ensure Chatbot Initialized ---

if "vector_store" not in globals():
    raise RuntimeError("❌ Vector store not found. Run vector store build cell first.")

if "llm" not in globals():
    llm = GeminiLLM()

if "chatbot" not in globals():
    if "df_cleaned" in globals():
        chatbot = RAGChatbot(vector_store, llm, config, df_cleaned)
    elif "df_chunked" in globals():
        chatbot = RAGChatbot(vector_store, llm, config, df_chunked)
    else:
        raise RuntimeError("❌ No dataframe found for chatbot initialization.")
    
    print("✅ Chatbot auto-initialized successfully")


✅ Chatbot auto-initialized successfully


### CELL 12: Interactive Chat Interface ###

In [33]:
from IPython.display import display, HTML, clear_output
import ipywidgets as widgets

# Chat history
chat_history = []

# Create UI components
output_area = widgets.Output()
question_input = widgets.Text(
    placeholder='Ask about restaurants (e.g., "Best food quality?")',
    description='Question:',
    layout=widgets.Layout(width='70%')
)
restaurant_input = widgets.Text(
    placeholder='Optional: Filter by restaurant name',
    description='Restaurant:',
    layout=widgets.Layout(width='70%')
)
send_button = widgets.Button(
    description='Send',
    button_style='primary',
    icon='paper-plane'
)
clear_button = widgets.Button(
    description='Clear Chat',
    button_style='warning',
    icon='trash'
)

def format_chat_message(role, message, sources=None):
    """Format chat message with styling"""
    if role == "user":
        return f'''
        <div style="background: #e3f2fd; padding: 10px; margin: 5px 0; border-radius: 10px; border-left: 4px solid #2196F3;">
            <strong>🧑 You:</strong> {message}
        </div>
        '''
    else:
        return f"""
        <div style='background: #f5f5f5; padding: 10px; margin: 5px; border-radius: 10px; border-left: 4px solid #6C63FF;'>
            <strong>🤖 Assistant:</strong><br>
            {message}
        </div>
        """

def send_message(b):
    """Handle send button click"""
    question = question_input.value.strip()
    restaurant = restaurant_input.value.strip() if restaurant_input.value.strip() else None

    if not question:
        with output_area:
            clear_output(wait=True)
            for msg in chat_history:
                display(HTML(msg))
            display(HTML('<p style="color: red;">⚠️ Please enter a qquestion</p>'))
        return

    # Add user message to history
    user_msg = format_chat_message("user", question + (f" (Restaurant: {restaurant})" if restaurant else ""))
    chat_history.append(user_msg)

    # Get response
    result = chatbot.query(question, restaurant_filter=restaurant)  

    # Add bot response to history
    bot_msg = format_chat_message(
        "assistant",
        result["answer"] + f"\n\nConfidence: {result['confidence']}"
    )

    chat_history.append(bot_msg)

    # Update display
    with output_area:
        clear_output(wait=True)
        for msg in chat_history:
            display(HTML(msg))

    # Clear inputs
    question_input.value = ""
    restaurant_input.value = ""

def clear_chat(b):
    """Clear chat history"""
    global chat_history
    chat_history = []
    with output_area:
        clear_output()
        display(HTML('<p style="color: #666;">Chat cleared. Start a new conversation!</p>'))

# Attach event handlers
send_button.on_click(send_message)
clear_button.on_click(clear_chat)
question_input.on_submit(lambda x: send_message(None))

# Display UI
print("\n" + "="*80)
print("🤖 INTERACTIVE RESTAURANT CHATBOT")
print("="*80)
print("\nAsk questions about restaurants or get recommendations!")  
print("\nExample questions:")
print("  - Which restaurant has the best food?")
print("  - Tell me about the service quality")
print("  - Is Izumi Japanese Kitchen good for couples?")
print("  - What do people say about prices?")
print("\n" + "="*80 + "\n")

display(widgets.VBox([
    widgets.HTML("<h3>🍽️ Restaurant Intelligence Chatbot</h3>"),    
    question_input,
    restaurant_input,
    widgets.HBox([send_button, clear_button]),
    output_area
]))


🤖 INTERACTIVE RESTAURANT CHATBOT

Ask questions about restaurants or get recommendations!

Example questions:
  - Which restaurant has the best food?
  - Tell me about the service quality
  - Is Izumi Japanese Kitchen good for couples?
  - What do people say about prices?




In [ ]:
docs = vector_store.similarity_search("KFC", k=3)

for d in docs:
    print(d.metadata)
    print("------")


In [ ]:
entities = chatbot.extract_entities("Tell me about Pallabi Hotel & Restaurant")

docs = chatbot.hybrid_retrieval(
    "Tell me about Pallabi Hotel & Restaurant",
    entities,
    k=6
)

print("Total docs:", len(docs))

for d in docs:
    print(d.metadata["restaurant"])


# # # Data Export for FastAPI  

In [ ]:
import os
import shutil

# Create data directory
data_dir = "data"
os.makedirs(data_dir, exist_ok=True)

# Export processed DataFrame to CSV
output_path = os.path.join(data_dir, "processed_reviews.csv")
df_cleaned.to_csv(output_path, index=False)
print(f"✅ Exported {len(df_cleaned)} reviews to {output_path}")

# Export as pickle (faster loading)
pickle_path = os.path.join(data_dir, "processed_reviews.pkl")
df_cleaned.to_pickle(pickle_path)
print(f"✅ Exported pickle to {pickle_path}")

# Copy vector database
source_vector_db = "/kaggle/working/restaurant_vector_db"
target_vector_db = "restaurant_vector_db"

if os.path.exists(source_vector_db):
    if os.path.exists(target_vector_db):
        shutil.rmtree(target_vector_db)
    shutil.copytree(source_vector_db, target_vector_db)
    print(f"✅ Copied vector database to {target_vector_db}")
else:
    print("⚠️ Vector database not found")

print("\n🎉 Data export complete!")
print("\nFiles created:")
print("  - data/processed_reviews.csv")
print("  - data/processed_reviews.pkl")
print("  - restaurant_vector_db/ (folder)")

In [ ]:
import os
import shutil
from langchain.vectorstores import Chroma
from chromadb.config import Settings

# Target Path
VECT_DB_PATH = "/kaggle/working/restaurant_vector_db"

print(f"🎯 Force building at: {VECT_DB_PATH}")

# 1. Clean up
if os.path.exists(VECT_DB_PATH):
    shutil.rmtree(VECT_DB_PATH)

# 2. Get documents and embeddings manually
print("📄 Preparing documents...")
vector_manager = VectorStoreManager(config)
documents = vector_manager.create_documents(df_chunked)
embeddings = vector_manager.embeddings

# 3. Direct Chroma Creation (Bypassing Manager)
print("🏗️ Creating ChromaDB directly...")
vector_store = Chroma.from_documents(
    documents=documents,
    embedding=embeddings,
    persist_directory=VECT_DB_PATH,
    client_settings=Settings(anonymized_telemetry=False)
)

# 4. Force Persist (IMPORTANT for older Chroma versions)
try:
    vector_store.persist()
    print("💾 Forced persistence called.")
except:
    print("ℹ️ Auto-persistence enabled (newer Chroma version).")

# 5. Verify & Zip
if os.path.exists(VECT_DB_PATH):
    print(f"✅ SUCCESS! Folder created: {os.listdir(VECT_DB_PATH)}")
    shutil.make_archive("restaurant_vector_db", 'zip', root_dir="/kaggle/working", base_dir="restaurant_vector_db")
    print("🎉 'restaurant_vector_db.zip' is ready to download!")
else:
    print("❌ STILL FAILED. ChromaDB is refusing to write to this path.")